In [2]:
import sys
assert sys.version_info >= (3, 7)

import numpy as np
import cv2 as cv
from util_func import *
import scipy

## Exercise 1

In [39]:
#modify transformation matrix
img = cv.imread("images/lena.jfif")

h, w = img.shape[:2]
yc, xc = h //2, w //2
angle = -45 #positive -> rotate counterclockwise

#get transformaton matrix
M = cv.getRotationMatrix2D((xc, yc), angle, 1)

#M = |cos(theta) -sin(theta) tx|
#    |sin(theta) cos(theta)  ty|
cosine = abs(M[0, 0])
sine = abs(M[0, 1])

#x' = ax + by [a=>scalling component in x-direction | b=> rotation comp in x-direction]
#y' = cx + dy 
new_w = int((h * sine) + (w * cosine)) # h*sine => anount of distance extends vertically | w*cosine => horizontally | sum to get total width
new_h = int((h * cosine) + (w * sine)) # h*cosine => vertical | w*sine => horizontal

M[0, 2] += (new_w / 2) - xc #move img back to center
M[1, 2] += (new_h / 2) - yc

dst = cv.warpAffine(img, M, (new_w, new_h))

show_img("rotation", dst)

In [34]:
# use imutils
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25872 sha256=1968b355f30b158133d5741925514fdc2817af897c58ab8d545992c180d97aa3
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils


In [38]:
import imutils
img = cv.imread("images/lena.jfif")
rotated_image = imutils.rotate_bound(img, angle=45)
show_img("rotation", rotated_image)

## Exercise 2

In [29]:
bee_img = cv.imread("images/native-bee.png")
flower_img = cv.imread("images/flower.jfif")

# Define the position to place the flower in the bee image
x_pos = 5 
y_pos = 5  

flower_height, flower_width = flower_img.shape[:2]

# Convert flower image to grayscale
gray_flower = cv.cvtColor(flower_img, cv.COLOR_BGR2GRAY)

# Threshold the grayscale flower image to create a mask
mask = cv.threshold(gray_flower, 70, 255, cv.THRESH_BINARY)[1]

# Invert the mask to keep only non-black pixels
mask_inv = cv.bitwise_not(mask)

# Extract the region of interest (ROI) from the bee image
roi = bee_img[y_pos:y_pos + flower_height, x_pos:x_pos + flower_width]

# Use the mask to keep only the flower pixels and blend it with the ROI
roi_result = cv.bitwise_and(roi, roi, mask=mask_inv)
flower_result = cv.bitwise_and(flower_img, flower_img, mask=mask)
result = cv.add(roi_result, flower_result)

# Place the modified ROI back into the original bee image
bee_img[y_pos:y_pos + flower_height, x_pos:x_pos + flower_width] = result

show_img("result",bee_img)

## Exercise 3

In [51]:
kernel3 = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
kernel5 = np.full((5,5), -1)
kernel5[3,3] = 25

img = cv.imread("images/native-bee.png")
dst3 = cv.filter2D(img, -1, kernel3)
dst5 = cv.filter2D(img, -1, kernel5)

cv.imshow("orginal", img)
cv.imshow("3x3 kernel", dst3)
show_img("5x5 kernel", dst5)

## Exercise 4

In [55]:
img = cv.imread("images/noise_lena.jpg")

#average filter (cv.blur)
avg = cv.blur(img, (5, 5))

#Gaussian kernel => almost same as avg filter, blur effect more satisfactory
gaussian = cv.GaussianBlur(img, (5,5), 0)

#Median filter => good for removing salt and pepper noise
median = cv.medianBlur(img, 5)

cv.imshow("original", img)
cv.imshow("average", avg)
cv.imshow("gaussian", gaussian)
show_img("median", median)